In [ ]:
# install cloudant
!pip install cloudant

In [ ]:
from cloudant.client import Cloudant
username = "<< cloudant username >>"
password = "<< cloudant password >>"
url_string = "<< cloudant url >>"
client = Cloudant(username, password, url=url_string, connect=True, auto_renew=True)

In [ ]:
# Create a database using an initialized client
# The result is a new CloudantDatabase
my_database = client.create_database('<< cloudant db name >>')

# You can check that the database exists
if my_database.exists():
    print('Successfully created database!!')

In [ ]:
import os
from pathlib import Path

pathStr = '<< local absolute path to files >>'
path = Path(pathStr)

filenames_list = []

#Load word, html and pdf files only

docFiles_list = list(path.glob('**/*.doc'))
for file in docFiles_list:
    fileName = file.name
    filenames_list.append(fileName)
docFiles_list = filenames_list
print("docFiles_list list size is ", len(docFiles_list))
offset = len(filenames_list)
    
docxFiles_list = list(path.glob('**/*.docx'))
for file in docxFiles_list:
    fileName = file.name
    filenames_list.append(fileName)
docxFiles_list = filenames_list[offset:]
print("docxFiles_list list size is ", len(docxFiles_list))
offset = len(filenames_list)

pdfFiles_list = list(path.glob('**/*.pdf'))
for file in pdfFiles_list:
    fileName = file.name
    filenames_list.append(fileName)
pdfFiles_list =  filenames_list[offset:]
print("pdfFiles_list list size is ", len(pdfFiles_list))
offset = len(filenames_list)

htmlFiles_list = list(path.glob('**/*.html'))
for file in htmlFiles_list:
    fileName = file.name
    filenames_list.append(fileName)
htmlFiles_list =  filenames_list[offset:]
print("htmlFiles_list list size is ", len(htmlFiles_list))


In [ ]:
allFilesList = [docFiles_list, docxFiles_list, pdfFiles_list, htmlFiles_list]
contentTypes = ["application/msword", "application/vnd.openxmlformats-officedocument.wordprocessingml.document", "application/pdf", "text/html"]

In [ ]:
import time
from cloudant.document import Document
contentTypeIdx = 0
for fileslist in allFilesList:
    for docName in fileslist:
        # Upon entry into the document context, fetches the document from the
        # remote database, if it exists. Upon exit from the context, saves the
        # document to the remote database with changes made within the context.
        with Document(my_database, docName) as document:
            # The document is fetched from the remote database
            # Changes are made locally
            document['filename'] = docName
            # The document is saved to the remote database
            # Add attachment
            #document.put_attachment(docName, 'type/binary', data, headers=None)
            fileFullPath = pathStr + "/" + docName
            fileSize = os.path.getsize(fileFullPath)//1000000 #(in MB)

            sleep_time = fileSize * 2
            with open(fileFullPath, 'rb') as f:
                data = f.read()
                try:
                    time.sleep(sleep_time)
                    resp = document.put_attachment(docName, contentTypes[contentTypeIdx], data)
                    print("Successfully uploaded ", docName)
                    time.sleep(sleep_time)
                except Exception as e:
                    print("Failed to upload ", docName, str(e))
    contentTypeIdx = contentTypeIdx+1
    print("Files processed: ", contentTypeIdx)
